# Advent of Code 2025

> Computer science is no more about computers than astronomy is about telescopes.
>
-- Edsger W. Dijkstra

## Imports and definitions

In [1]:
from urllib import request
from itertools import accumulate, islice


def aocin(day):
    try:
        with open(f'input/{day}') as f:
            return f.read().strip()
    except FileNotFoundError:
        r = request.Request(f'https://adventofcode.com/2025/day/{day}/input')
        r.add_header('Cookie', open('../.aoccookie').read().strip())
        r.add_header('User-Agent', 'github.com/edoannunziata/jardin')
        with open(f'input/{day}', 'bw') as f:
            f.write(request.urlopen(r).read())
        with open(f'input/{day}') as f:
            return f.read().strip()

## [Day 1: Secret Entrance](https://adventofcode.com/2025/day/1)

In [2]:
rotations = [int(x.strip('R').replace('L', '-')) for x in aocin(1).split()]

A = sum(1 for x in accumulate(rotations, initial=50) if x % 100 == 0)
assert A == 992


def hits(move, start):
    revolutions = abs(move) // 100
    signed_mod = move % 100 if move >= 0 else -(-move % 100)
    return revolutions + (start != 0 and not 0 < start + signed_mod < 100)

A = sum(hits(r, s % 100) for r, s in zip(rotations, accumulate(rotations, initial=50)))
assert A == 6133